In [20]:
from mesa import Agent, Model
from mesa.time import SimultaneousActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

import numpy as np
import pandas as pd

import time
import datetime


In [21]:
class CleaningRobot(Agent):
    def __init__ (self, id, model, num_vaccums, dirty_percentage):
        super().__init__(id, model)
        self.num_vaccums = num_vaccums
        self.dirty_percentage = dirty_percentage
        
    def step(self):
        neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False)
        if len(neighbors) > 0:
            self.model.grid.move_to_empty(self)
        else:
            # Cleans the cell if its dirty
            if self.model.grid[self.pos[0]][self.pos[1]] == 1:
                #Cleans the cell
                self.model.grid[self.pos[0]][self.pos[1]] = 0  

            # Moves to a empty cell
            self.model.grid.move_to_empty(self)

In [22]:
class CleaningRobot(Model):
    def __init__ (self, width, height, num_vaccums, dirty_percentage):
        self.num_vaccums = num_vaccums
        self.dirty_cells = int((width * height) * (dirty_percentage / 100))
        self.schedule = SimultaneousActivation(self)
        self.space = MultiGrid(width, height, True)

        #Initializes vacuums in cell [1,1]
        for i in range(self.num_vaccums):
            agent = CleaningRobot(i, self)
            self.schedule.add(agent)
            self.grid.place_agent(agent, (1,1))

        #Initializes dirty cells
        for cell in self.dirty_cells:
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid[x][y] = 1

        #Gets the position of the agents
        self.datacollector = DataCollector(
            model_reporters={"Dirty Cells": "dirty_cells"},
            agent_reporters={"Position": "pos"}
        )
    
    def step(self):
        #Collects data from the agents defined in the reporters
        self.datacollector.collect(self)
        #Moves to the next step of the simulation
        self.schedule.step()

In [23]:
WIDTH = 100
HEIGHT = 100
NUM_VACCUMS = 1
DIRTY_PERCENTAGE = 90
MAX_STEPS = 100

start_time = time.time()
model = CleaningRobot(WIDTH, HEIGHT, NUM_VACCUMS, DIRTY_PERCENTAGE)
for i in range(MAX_STEPS):
    model.step()

#Print execution time
print('Execution Time:', str(datetime.timedelta(seconds=(time.time() - start_time))))


TypeError: CleaningRobot.__init__() missing 2 required positional arguments: 'num_vaccums' and 'dirty_percentage'

In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [25]:
#DUDAS
#Como hago la simulacion/animacion
#Como corrijo el error
#Como imprimo el porcentaje de celdas sucias al terminar la simulacion
#Como se como hacer la limpieza lo mas optima posible